# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model as lm
import matplotlib.pyplot as plt

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int,\
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float,\
              'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int,\
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv("kc_house_data.csv", dtype = dtype_dict)
train_data = pd.read_csv("kc_house_train_data.csv", dtype = dtype_dict)
test_data = pd.read_csv("kc_house_test_data.csv", dtype = dtype_dict)

In [3]:
 sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [4]:
example_features = train_data[['sqft_living','bedrooms','bathrooms']]
#print(example_features)


In [5]:
example_features.head()

,sqft_living,bedrooms,bathrooms
0,1180.0,3.0,1.00
1,2570.0,3.0,2.25
2,770.0,2.0,1.00
3,1960.0,4.0,3.00
4,1680.0,3.0,2.00


In [6]:
example_model = lm.LinearRegression()
example_model.fit(example_features,train_data[['price']])

LinearRegression()

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [7]:
print("regression intercept", example_model.intercept_)
print("regression coefficients", example_model.coef_)

regression intercept [87912.86581496]
regression coefficients [[   315.40669062 -65081.88711588   6942.16598637]]


Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [8]:
#example_weight_summary = example_model.coefficients
#print example_weight_summary

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [9]:
#example_predictions = example_model.predict(train_data)
#print example_predictions[0] # should be 271789.505878

example_predictions = example_model.predict(train_data[['sqft_living','bedrooms','bathrooms']])
print(example_predictions[0])

[271789.26537997]


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [10]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predicted = model.predict(data)
    # Then compute the residuals/errors
    residuals = predicted - outcome
    # Then square and add them up
    RSS = np.sum(np.square(residuals))
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [11]:
#rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
#print(rss_example_train) # should be 2.7376153833e+14
print(get_residual_sum_of_squares(example_model, test_data[['sqft_living', 'bedrooms', 'bathrooms']], test_data[['price']]))

price    2.737619e+14
dtype: float64


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [12]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [13]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [14]:
# create the remaining 3 features in both TEST and TRAIN data


train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] +  train_data['long']
test_data['lat_plus_long'] = test_data['lat'] +  test_data['long'] 


In [15]:
print(test_data[['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']].mean())

bedrooms_squared    12.446678
bed_bath_rooms       7.503902
log_sqft_living      7.550275
lat_plus_long      -74.653334
dtype: float64


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [16]:
train_data.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,0,98178,47.5112,-122.257,1340.0,5650.0,9.0,3.00,7.073270,-74.7458
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,1991,98125,47.7210,-122.319,1690.0,7639.0,9.0,6.75,7.851661,-74.5980
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,0,98028,47.7379,-122.233,2720.0,8062.0,4.0,2.00,6.646391,-74.4951
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,0,98136,47.5208,-122.393,1360.0,5000.0,16.0,12.00,7.580700,-74.8722
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,0,98074,47.6168,-122.045,1800.0,7503.0,9.0,6.00,7.426549,-74.4282


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [17]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [18]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = lm.LinearRegression()
model_1.fit(train_data[model_1_features],train_data[['price']])

model_2 = lm.LinearRegression()
model_2.fit(train_data[model_2_features],train_data[['price']])

model_3 = lm.LinearRegression()
model_3.fit(train_data[model_3_features],train_data[['price']])

LinearRegression()

In [19]:
# Examine/extract each model's coefficients:
print("regression intercept", model_1.intercept_)
print("regression coefficients", model_1.coef_)
print("-------------------------------------------------------------------------------------")
print("regression intercept", model_2.intercept_)
print("regression coefficients", model_2.coef_)
print("-------------------------------------------------------------------------------------")
print("regression intercept", model_3.intercept_)
print("regression coefficients", model_3.coef_)

regression intercept [-69075726.79256989]
regression coefficients [[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
  -3.09374351e+05]]
-------------------------------------------------------------------------------------
regression intercept [-66867968.87107899]
regression coefficients [[ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
  -2.94298969e+05  2.55796520e+04]]
-------------------------------------------------------------------------------------
regression intercept [-62036084.98609832]
regression coefficients [[ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
  -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
   1.27334900e+05]]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [20]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model_1 = get_residual_sum_of_squares(model_1, train_data[model_1_features], train_data[['price']])
rss_model_2 = get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data[['price']])
rss_model_3 = get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data[['price']])
print(rss_model_1)
print(rss_model_2)
print(rss_model_3)

price    9.678800e+14
dtype: float64
price    9.584196e+14
dtype: float64
price    9.034365e+14
dtype: float64


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [21]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1 = get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data[['price']])
rss_model_2 = get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data[['price']])
rss_model_3 = get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data[['price']])
print(rss_model_1)
print(rss_model_2)
print(rss_model_3)

price    2.255005e+14
dtype: float64
price    2.233775e+14
dtype: float64
price    2.592363e+14
dtype: float64


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [22]:
np.linspace(-10, 10, 100)


array([-10.        ,  -9.7979798 ,  -9.5959596 ,  -9.39393939,
        -9.19191919,  -8.98989899,  -8.78787879,  -8.58585859,
        -8.38383838,  -8.18181818,  -7.97979798,  -7.77777778,
        -7.57575758,  -7.37373737,  -7.17171717,  -6.96969697,
        -6.76767677,  -6.56565657,  -6.36363636,  -6.16161616,
        -5.95959596,  -5.75757576,  -5.55555556,  -5.35353535,
        -5.15151515,  -4.94949495,  -4.74747475,  -4.54545455,
        -4.34343434,  -4.14141414,  -3.93939394,  -3.73737374,
        -3.53535354,  -3.33333333,  -3.13131313,  -2.92929293,
        -2.72727273,  -2.52525253,  -2.32323232,  -2.12121212,
        -1.91919192,  -1.71717172,  -1.51515152,  -1.31313131,
        -1.11111111,  -0.90909091,  -0.70707071,  -0.50505051,
        -0.3030303 ,  -0.1010101 ,   0.1010101 ,   0.3030303 ,
         0.50505051,   0.70707071,   0.90909091,   1.11111111,
         1.31313131,   1.51515152,   1.71717172,   1.91919192,
         2.12121212,   2.32323232,   2.52525253,   2.72